In [ ]:
pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# Setup WebDriver
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)

# Open URL
url = 'https://www.psx.com.pk/market-summary/'
driver.get(url)
time.sleep(3)

# Scroll to load dynamic content
scroll_pause_time = 0.5
scroll_step = 300
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    for i in range(0, last_height, scroll_step):
        driver.execute_script(f"window.scrollTo(0, {i});")
        time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

print("✅ Finished scrolling")

# Parse the page
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# Find all market sections
sections = soup.select("div.table-responsive")

# Store data in memory
psx_data = []

for section in sections:
    thead = section.find('thead')
    heading_tag = thead.find('h4') if thead else None
    heading = heading_tag.text.strip() if heading_tag else "UNKNOWN CATEGORY"

    table = section.find('table')
    tbody = table.find('tbody')
    rows = tbody.find_all('tr') if tbody else []

    for row in rows:
        cols = row.find_all('td')
        if len(cols) >= 8:
            scrip = cols[0].text.strip()
            ldcp = cols[1].text.strip()
            open_ = cols[2].text.strip()
            high = cols[3].text.strip()
            low = cols[4].text.strip()
            current = cols[5].text.strip()
            change = cols[6].text.strip()
            volume = cols[7].text.strip()

            psx_data.append([heading, scrip, ldcp, open_, high, low, current, change, volume])

# Optional: print preview
for row in psx_data[:5]:
    print(row)


✅ Finished scrolling
['AUTOMOBILE ASSEMBLER', 'SCRIP', 'LDCP', 'OPEN', 'HIGH', 'LOW', 'CURRENT', 'CHANGE', 'VOLUME']
['AUTOMOBILE ASSEMBLER', 'AL-Ghazi Tractors', '388.58', '386.11', '427.44', '386.11', '427.44', '38.86', '63,412']
['AUTOMOBILE ASSEMBLER', 'Atlas Honda Ltd', '1,149.99', '1157.7', '1165.0', '1144.0', '1144.98', '-4.16', '6,899']
['AUTOMOBILE ASSEMBLER', 'Dewan Motors', '37.25', '37.48', '37.48', '36.2', '36.29', '-0.91', '1,878,172']
['AUTOMOBILE ASSEMBLER', 'Ghandhara Automobile', '478.30', '478.3', '479.8', '471.1', '472.99', '-5.83', '331,180']


In [2]:
import csv

with open("psx_market_data.csv", "w", newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["CATEGORY", "SCRIP", "LDCP", "OPEN", "HIGH", "LOW", "CURRENT", "CHANGE", "VOLUME"])
    writer.writerows(psx_data)

print("✅ Data written to 'psx_market_data.csv'")

✅ Data written to 'psx_market_data.csv'
